In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, CSVLogger
import matplotlib.pyplot as plt

C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("333836601.csv")
pd.set_option('display.max_columns', 100)
df = df.drop(['employees_created'], axis=1)
df = df.drop(['employee_id'], axis=1)
df["age"] = df["age"].fillna(df["age"].mean())
df["influx_route_midium"] = df["influx_route_midium"].fillna('null')
df["influx_route_small"] = df["influx_route_small"].fillna('null')

df_a = df.copy()
df_b = pd.get_dummies(df_a['influx_route_big'], drop_first = True)
df_c = pd.get_dummies(df_a['influx_route_midium'], drop_first = True)
df_d= pd.get_dummies(df_a['influx_route_small'], drop_first = True)
df_e= pd.get_dummies(df_a['device'], drop_first = True)

df = pd.concat([df, df_b, df_c, df_d, df_e], axis=1)
df = df.drop(['influx_route_big'], axis=1)
df = df.drop(['influx_route_midium'], axis=1)
df = df.drop(['influx_route_small'], axis=1)
df = df.drop(['device'], axis=1)

from sklearn.preprocessing import LabelEncoder
df_a = df.copy()

# Create a label encoder object
le = LabelEncoder()
le_count = 0

pd.set_option('display.max_columns', 100)
df.head()

,age,gender,entry_date_diffs,interview_date_diffs,open_count,mail_click,owned_visit,advertisement,career_change_desired_date_1,career_change_desired_date_2,career_change_desired_date_3,career_change_desired_date_4,now_status_1,now_status_2,now_status_3,now_status_4,now_status_5,contact_result_1,contact_result_2,contact_result_4,contact_result_5,objective_variable,DSP,SEO,twitter広告,その他,アフィリエイト,メルマガ,リスティング,提携媒体,GDN,GoogleMap,Google検索,KWターゲティング,LINE,TOP埋め込みEF,YDN,Yahoo検索,docomo,null,useful下部固定,webもり,アクセストレード,アフィリエイトB,キャリアパーク,フォロワーターゲティング,プレスコ,プロモツイート,メディパートナー,リマーケティング,...,infeed_remarketing,infeed_インタレストマッチ,infeed_ターゲティング,null,こだわり軸,セミナー,ターゲティング,ターゲティング_remarketing,ターゲティング_インタレストマッチ,ハローワーク軸,フォロワーターゲティング,フリーター軸,ミドルワード軸,メルマガ_登録半年以内_アプローチ+面談日程連絡待ち_求人紹介_転職無し,メルマガ_登録半年以内_アプローチ_ハタラクティブ魅力付け,メルマガ_登録半年以内_面談済み_就活ノウハウ_就活準備,メルマガ_登録半年以内_面談済み_求人紹介_残業無し推し,メルマガ_登録半年以前_面談済み_就活ノウハウ_就活全体,メルマガ_登録半年以前_面談済み_求人紹介_転職無し,メルマガ_面談済み_応募意思あり_CCテキストよびかけ,リターケティング,リマーケティング,友達紹介カード,地域軸,地域軸_未経験,大卒軸,就職軸_学歴KW軸,属性軸,属性軸_フリーター軸,属性軸_大卒軸,属性軸_既卒軸,属性軸_第二新卒軸,属性軸_高卒軸,手動プレースメント,指名検索軸,支援軸,既卒軸,正社員軸,相談軸,第二新卒軸,職種軸,自動プレースメント,自動メール_登録10日後経過面談未予約_カウンセラー紹介,自動メール_登録7日後経過面談未予約_就職成功事例紹介,興味関心ターゲティング,転職軸,雇用形態軸,面談,高卒軸,SP
0,18.0,1,17,11,0,0,5,0,0,1,0,0,1,0,0,0,0,1,0,0,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,29.0,0,5,5,6,7,37,1,1,0,0,0,1,0,0,0,0,1,0,1,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,25.0,0,16,16,1,0,8,0,0,0,1,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,25.0,1,2,2,1,0,22,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,21.0,0,0,0,3,2,15,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
df_train = df.copy()

#変数を配列へ変換
from sklearn.cross_validation import train_test_split

X = df_train.drop("objective_variable" , axis=1).values
y = df_train["objective_variable"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
df_train.shape

(12142, 121)

In [6]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=29, init='uniform', activation='relu'))
model.add(Dense(29, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=29, activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(29, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  """


In [7]:
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
hist = model.fit(X_train, y_train,
                 batch_size=10,
                 verbose=1,
                 nb_epoch=150,
                 validation_split=0.1,
                 callbacks=[early_stopping])

C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 7650 samples, validate on 850 samples
Epoch 1/150
7650/7650 [==============================] - 2s 197us/step - loss: 0.6482 - acc: 0.6216 - val_loss: 0.6187 - val_acc: 0.6776
Epoch 2/150
7650/7650 [==============================] - 1s 147us/step - loss: 0.6219 - acc: 0.6569 - val_loss: 0.5928 - val_acc: 0.6906
Epoch 3/150
7650/7650 [==============================] - 1s 134us/step - loss: 0.6143 - acc: 0.6686 - val_loss: 0.5952 - val_acc: 0.6847
Epoch 4/150
7650/7650 [==============================] - 1s 135us/step - loss: 0.6105 - acc: 0.6759 - val_loss: 0.5862 - val_acc: 0.7035
Epoch 5/150
7650/7650 [==============================] - 1s 132us/step - loss: 0.6073 - acc: 0.6788 - val_loss: 0.5858 - val_acc: 0.6929
Epoch 6/150
7650/7650 [==============================] - 1s 159us/step - loss: 0.6054 - acc: 0.6797 - val_loss: 0.5859 - val_acc: 0.6965
Epoch 7/150
7650/7650 [==============================] - 1s 135us/step - loss: 0.6042 - acc: 0.6763 - val_loss: 0.5835 - val_acc: 0

In [44]:
# fit the model
hist = model.fit(X_train, y_train, nb_epoch=150, batch_size=10)

C:\Users\ryosuke.nishioka\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/150
8501/8501 [==============================] - 2s 211us/step - loss: 0.6392 - acc: 0.6419
Epoch 2/150
8501/8501 [==============================] - 2s 188us/step - loss: 0.6139 - acc: 0.6729
Epoch 3/150
8501/8501 [==============================] - 2s 198us/step - loss: 0.6088 - acc: 0.6764
Epoch 4/150
8501/8501 [==============================] - 2s 263us/step - loss: 0.6041 - acc: 0.6826
Epoch 5/150
8501/8501 [==============================] - 2s 283us/step - loss: 0.6006 - acc: 0.6799
Epoch 6/150
8501/8501 [==============================] - 2s 243us/step - loss: 0.5977 - acc: 0.6860
Epoch 7/150
8501/8501 [==============================] - 2s 178us/step - loss: 0.5960 - acc: 0.6829
Epoch 8/150
8501/8501 [==============================] - 2s 231us/step - loss: 0.5938 - acc: 0.6858
Epoch 9/150
8501/8501 [==============================] - 2s 214us/step - loss: 0.5915 - acc: 0.6850
Epoch 10/150
8501/8501 [==============================] - 1s 173us/step - loss: 0.5909 - acc: 0.6874

In [17]:
score = model.evaluate(X_test, y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

3644/3644 [==============================] - 0s 30us/step
Test loss: 0.6091566130960551
Test accuracy: 0.6723380900764047


In [18]:
from sklearn import metrics as mtr

prob = model.predict_proba(X_test)
fpr, tpr, thresholds = mtr.roc_curve(y_test, prob, pos_label=1)
mtr.auc(fpr, tpr)

0.7155281983894591

In [20]:
class_predict = model.predict(X_test)
# 精度を計算してみます。
print('テストデータに対する正解率： %.4f' %mtr.accuracy_score(y_test,class_predict))
#print('テストデータに対する roc-auc： %.4f' %mtr.roc_auc_score(y_test, prob))
#print('テストデータに対する log-loss： %.4f' %mtr.log_loss(y_test, prob))

ValueError: Can't handle mix of binary and continuous

In [25]:
prob = model.predict_proba(X_test)
# 精度を計算してみます。
mtr.roc_auc_score(y_test, prob)
mtr.log_loss(y_test, prob)

0.609156612659785

In [ ]:
http://aidiary.hatenablog.com/entry/20161103/1478176348